# 因果効果推定手法の実装

LaLonde データセットを用いて、因果効果推定の主要な手法を実装・比較する。
- 回帰ベース（OLS）
- 傾向スコアマッチング（PSM）
- 逆確率重み付け（IPW）
- Doubly Robust 推定量


## 1. データの読み込みと前処理


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Load LaLonde dataset
df = pd.read_csv('data/input/ec675_nsw.tab', sep='\t')

print(f"Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nData types:\n{df.dtypes}")


Shape: (19204, 13)

Columns: ['treated', 'age', 'educ', 'black', 'married', 'nodegree', 'dwincl', 're74', 're75', 're78', 'hisp', 'early_ra', 'sample']

Data types:
treated     float64
age           int64
educ          int64
black         int64
married       int64
nodegree      int64
dwincl      float64
re74        float64
re75        float64
re78        float64
hisp          int64
early_ra    float64
sample      float64
dtype: object


In [2]:
# Handle missing values
df = df.dropna(subset=['treated', 're78'])

# Prepare variables
treatment = df['treated'].values
outcome = df['re78'].values

# Covariates (excluding treated and outcome)
X_cols = ['age', 'educ', 'black', 'married', 'nodegree', 're74', 're75']
X = df[X_cols].values

print(f"\nTreatment group size: {(treatment == 1).sum()}")
print(f"Control group size: {(treatment == 0).sum()}")
print(f"Mean outcome (treated): {outcome[treatment == 1].mean():.2f}")
print(f"Mean outcome (control): {outcome[treatment == 0].mean():.2f}")
print(f"Naive ATE: {outcome[treatment == 1].mean() - outcome[treatment == 0].mean():.2f}")



Treatment group size: 297
Control group size: 425
Mean outcome (treated): 5976.35
Mean outcome (control): 5090.05
Naive ATE: 886.30


## 2. 回帰ベースの因果効果推定


In [3]:
# OLS regression-based estimation
X_reg = np.column_stack([treatment, X])
model_ols = LinearRegression()
model_ols.fit(X_reg, outcome)
ate_ols = model_ols.coef_[0]

print(f"OLS-based ATE: {ate_ols:.2f}")


OLS-based ATE: 822.99


## 3. 傾向スコアの推定


In [4]:
# Estimate propensity scores
ps_model = LogisticRegression(max_iter=1000)
ps_model.fit(X, treatment)
propensity_scores = ps_model.predict_proba(X)[:, 1]

print(f"Propensity score range: [{propensity_scores.min():.4f}, {propensity_scores.max():.4f}]")
print(f"Treated group - mean PS: {propensity_scores[treatment == 1].mean():.4f}")
print(f"Control group - mean PS: {propensity_scores[treatment == 0].mean():.4f}")


Propensity score range: [0.2495, 0.5496]
Treated group - mean PS: 0.4177
Control group - mean PS: 0.4070


## 4. 傾向スコアマッチング（PSM）


In [5]:
# 1-to-1 nearest neighbor matching on propensity scores
treated_idx = np.where(treatment == 1)[0]
control_idx = np.where(treatment == 0)[0]

matched_pairs = []
caliper = 0.1
used_controls = set()

for t_idx in treated_idx:
    ps_t = propensity_scores[t_idx]
    ps_controls = propensity_scores[control_idx]
    
    # Find closest control within caliper
    distances = np.abs(ps_controls - ps_t)
    valid_controls = np.where(distances <= caliper)[0]
    
    if len(valid_controls) > 0:
        closest = valid_controls[np.argmin(distances[valid_controls])]
        c_idx = control_idx[closest]
        
        if c_idx not in used_controls:
            matched_pairs.append((t_idx, c_idx))
            used_controls.add(c_idx)

matched_pairs = np.array(matched_pairs)
ate_psm = (outcome[matched_pairs[:, 0]].mean() - 
           outcome[matched_pairs[:, 1]].mean())

print(f"Matched pairs: {len(matched_pairs)}")
print(f"PSM-based ATE: {ate_psm:.2f}")


Matched pairs: 181
PSM-based ATE: 737.63


## 5. 逆確率重み付け（IPW）


In [6]:
# IPW (Inverse Probability Weighting)
ps_clipped = np.clip(propensity_scores, 0.01, 0.99)

# Weights: treatment group: 1/PS, control group: 1/(1-PS)
weights = np.where(treatment == 1, 1 / ps_clipped, 1 / (1 - ps_clipped))

# Weighted ATE
treated_weighted = np.sum(weights[treatment == 1] * outcome[treatment == 1]) / np.sum(weights[treatment == 1])
control_weighted = np.sum(weights[treatment == 0] * outcome[treatment == 0]) / np.sum(weights[treatment == 0])
ate_ipw = treated_weighted - control_weighted

print(f"IPW-based ATE: {ate_ipw:.2f}")


IPW-based ATE: 816.11


## 6. Doubly Robust 推定量


In [7]:
# Fit regression models for outcome prediction
# Model for treated: E[Y|X, T=1]
treated_mask = treatment == 1
reg_treated = LinearRegression()
reg_treated.fit(X[treated_mask], outcome[treated_mask])
pred_y1 = reg_treated.predict(X)

# Model for control: E[Y|X, T=0]
control_mask = treatment == 0
reg_control = LinearRegression()
reg_control.fit(X[control_mask], outcome[control_mask])
pred_y0 = reg_control.predict(X)

# Doubly Robust estimator
ps_clipped = np.clip(propensity_scores, 0.01, 0.99)

# ATE_1 component: treated units
component1 = (treatment * (outcome - pred_y1) / ps_clipped) + pred_y1

# ATE_0 component: control units
component0 = ((1 - treatment) * (outcome - pred_y0) / (1 - ps_clipped)) + pred_y0

ate_dr = component1.mean() - component0.mean()

print(f"Doubly Robust ATE: {ate_dr:.2f}")


Doubly Robust ATE: 818.15


## 7. 結果の比較


In [8]:
# Summary of results
results = {
    'Method': ['Naive (Unadjusted)', 'Regression-based', 
               'Propensity Score Matching', 'IPW', 'Doubly Robust'],
    'ATE': [outcome[treatment == 1].mean() - outcome[treatment == 0].mean(),
            ate_ols, ate_psm, ate_ipw, ate_dr]
}

results_df = pd.DataFrame(results)
print("=" * 60)
print("Causal Effect Estimation Results")
print("=" * 60)
print(results_df.to_string(index=False))


Causal Effect Estimation Results
                   Method        ATE
       Naive (Unadjusted) 886.303822
         Regression-based 822.991814
Propensity Score Matching 737.628977
                      IPW 816.111031
            Doubly Robust 818.151493
